# Introduction
This notebook illustrates how to perform Random Forest classification of a Sentinel-2 image using [Remotior Sensus
](https://fromgistors.blogspot.com/p/remotior-sensus.html).

> **Remotior Sensus** *(which is Latin for “a more remote sense”) is a Python package that allows for the processing of remote sensing images and GIS data*.
*Other tutorials are available [here](https://remotior-sensus.readthedocs.io/en/latest/basic_tutorials.html).*



# Install Remotior Sensus and start a new Session

First, we install Remotior Sensus using Pip.

In [1]:
!pip install -U remotior_sensus


We import Remotior Sensus and **start a Remotior Sensus’** [**session**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.core.session.html), defining the number of processes (`n_processes`) and RAM (`available_ram`) available in the system.

>*For further information about sessions, please read the [User Manual](https://remotior-sensus.readthedocs.io).*

In [2]:
import remotior_sensus

rs = remotior_sensus.Session(n_processes=2, available_ram=3640)

INFO[2023-12-08T19:46:41.210] system_tools.get_system_info[38] version:0.1.24; system: Linux; 64bit: True; n_processes: 2; ram: 3640; temp.dir: /tmp/remotior_sensus_12081jxz9hxk
WARNING[2023-12-08T19:46:41.213] session._check_dependencies[457] No module named 'torch'


⚠ warning: dependency error: pytorch


# Prepare Input Data

For the classification we need to:

1. Create a BandSet using an image
2. Load a training input
3. Perform the random forest classification

A [**BandSet**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.core.bandset_catalog.html#remotior_sensus.core.bandset_catalog.BandSet) is an object that includes information about bands (from the file path to the spatial and spectral characteristics).

We can define a working directory where data of this tutorial are saved.

In [3]:
working_directory = "mystorage"


Before creating the BandSet we should prepare input data.
For the purpose of this tutorial, in order to reduce computation time, we are going to download a Copernicus Sentinel-2 image which includes the following bands.

| Sentinel-2 Bands                    | Central Wavelength  [micrometers]   |  Resolution [meters]   |
|-------------------------------------|-------------------------------------|------------------------|
| Band 2 - Blue                       | 0.490                               |  10                    |
| Band 3 - Green                      |  0.560                              |  10                    |
| Band 4 - Red                        | 0.665                               |  10                    |
| Band 5 - Vegetation Red Edge        | 0.705                               |  20                    |
| Band 6 - Vegetation Red Edge        | 0.740                               |  20                    |
| Band 7 - Vegetation Red Edge        | 0.783                               |  20                    |
| Band 8 - NIR                        | 0.842                               |  10                    |
| Band 8A - Vegetation Red Edge       | 0.865                               |  20                    |
| Band 11 - SWIR                      | 1.610                               |  20                    |
| Band 12 - SWIR                      | 2.190                               |  20                    |


## Image Search

For downloading Copernicus images we need to enter user and password of the https://dataspace.copernicus.eu.


In [4]:
from getpass import getpass
import certifi
import os

# Provide CDSE account credentials - replace with your own data
user = os.environ["CDSE_USERNAME"]
password = os.environ["CDSE_PASSWORD"]

# user = getpass('Copernicus user: ')
# password = getpass('Copernicus password: ')

Copernicus user:  ········
Copernicus password:  ········



We use the function [**download_products.search**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.tools.download_products.html#remotior_sensus.tools.download_products.search) to **query the Sentinel-2 database** and retrieve a table of available images:
* `name_filter` allows for searching images that match a **tile name**
* `date_from` and `date_to` set a period range
* `max_cloud_cover` sets a threshold for **maximum cloud cover**
* `result_number` returns a limited number of results, in this case 1 image

> For more information about how to download data using Remotior Sensus please read
[this tutorial
](https://remotior-sensus.readthedocs.io/en/latest/tutorial_ndvi.html).

In [5]:
query_result = rs.download_products.search(
    product="Sentinel-2",
    name_filter="32TPR",
    date_from="2023-07-01",
    date_to="2023-07-30",
    max_cloud_cover=10,
    result_number=1,
    copernicus_user=user,
    copernicus_password=password,
)

INFO[2023-12-08T19:46:52.080] download_products.query_sentinel_2_database[114] start
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): identity.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://identity.dataspace.copernicus.eu:443 "POST /auth/realms/CDSE/protocol/openid-connect/token HTTP/1.1" 200 3354


 search [ 90%]:search in progress ⚙

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): catalogue.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://catalogue.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(MTD_MSIL2A.xml)/$value HTTP/1.1" 301 162
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): zipper.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://zipper.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(MTD_MSIL2A.xml)/$value HTTP/1.1" 200 54928
DEBUG:urllib3.connectionpool:https://zipper.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(MTD_MSIL2A.xml)/$value HTTP/1.1" 200 54928
DEBUG:urllib3.connectionpool:S

 search [100%]: ⬤

The query_result is an object OutputManager including a product table (NumPy recarray) that we can retrive with the attribute extra['product_table
We can have a look at the **table fields**, which include information such as acquisition date, cloud cover, and a preview link. For instance, we can read the records of the first image in the table.

In [6]:
product_table = query_result.extra["product_table"]
print(print(product_table[0].pprint()))

         product: Sentinel-2
           image: L2A_T32TPR_A033081_20230707T101428
      product_id: S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302
acquisition_date: 2023-07-07
     cloud_cover: 3
       zone_path: 32TPR
             row: None
         min_lat: 45.034190140253
         min_lon: 10.2700756717775
         max_lat: 46.0462654563034
         max_lon: 11.7105441475977
      collection: None
            size: 1
         preview: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(GRANULE)/Nodes(L2A_T32TPR_A033081_20230707T101428)/Nodes(QI_DATA)/Nodes(T32TPR_20230707T100559_PVI.jp2)/$value
             uid: 302382f3-fb33-4a3a-a5db-5836451b374e
None


We can display an image preview.
`product_table.preview[0]` is the url of the preview of the first product in the table.

In [7]:
access_token, session_state = rs.download_products.get_copernicus_token(
    user=user, password=password
)
temp_file = rs.configurations.temp.temporary_file_path(name_suffix=".jp2")
check = rs.configurations.multiprocess.multi_download_file(
    url_list=[product_table.preview[0]],
    output_path_list=[temp_file],
    access_token=access_token,
    copernicus=True,
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): identity.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://identity.dataspace.copernicus.eu:443 "POST /auth/realms/CDSE/protocol/openid-connect/token HTTP/1.1" 200 3354
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): catalogue.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://catalogue.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(GRANULE)/Nodes(L2A_T32TPR_A033081_20230707T101428)/Nodes(QI_DATA)/Nodes(T32TPR_20230707T100559_PVI.jp2)/$value HTTP/1.1" 301 162
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): zipper.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://zipper.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)

In [ ]:
# @title Figure
from osgeo import gdal
import numpy
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

raster = gdal.Open(temp_file)
blue_band = raster.GetRasterBand(1).ReadAsArray()
green_band = raster.GetRasterBand(2).ReadAsArray()
red_band = raster.GetRasterBand(3).ReadAsArray()
image = numpy.stack((red_band, green_band, blue_band), axis=-1)
plt.imshow(image)
plt.rcParams["figure.figsize"] = (15, 15)
plt.axis("off")
plt.show()
band = image = plt = None

To download the images, we use the function [**download_products.download**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.tools.download_products.html#remotior_sensus.tools.download_products.download) and download the 10m and 20m bands:
* `product_table` sets the images to be downloaded from the previously query results (in this case one image)
* `band_list` defines a list of band names which for Sentinel-2 are: `'02', '03', '04', '05', '06', '07', '08', '8a', '11', '12'`.
* `output_path` defines where downloaded files are saved.

During the download, a directory inside the `output_directory` is created for the downloaded image.

> `downloaded_image` is an an object [**OutputManager**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.core.output_manager.html) that includes a list of created directories in the attribute `extra['directory_paths']`.

In [9]:
downloaded_image = rs.download_products.download(
    product_table=product_table,
    band_list=["02", "03", "04", "05", "06", "07", "08", "8A", "11", "12"],
    output_path=working_directory + "/downloaded_images",
    copernicus_user=user,
    copernicus_password=password,
)

print(downloaded_image.extra["directory_paths"])

INFO[2023-12-08T19:48:28.451] download_products.download[542] start
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): identity.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://identity.dataspace.copernicus.eu:443 "POST /auth/realms/CDSE/protocol/openid-connect/token HTTP/1.1" 200 3354
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): catalogue.dataspace.copernicus.eu:443


 download products [  0%]:starting ⚙

DEBUG:urllib3.connectionpool:https://catalogue.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(MTD_MSIL2A.xml)/$value HTTP/1.1" 301 162
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): zipper.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://zipper.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(MTD_MSIL2A.xml)/$value HTTP/1.1" 200 54928
DEBUG:urllib3.connectionpool:https://zipper.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(MTD_MSIL2A.xml)/$value HTTP/1.1" 200 54928
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): catalogue.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:h

 download products [  0%] starting ⚙

DEBUG:urllib3.connectionpool:https://zipper.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(GRANULE)/Nodes(L2A_T32TPR_A033081_20230707T101428)/Nodes(IMG_DATA)/Nodes(R10m)/Nodes(T32TPR_20230707T100559_B02_10m.jp2)/$value HTTP/1.1" 200 119714873


...

 download products [ 86%] [elapsed 2min04sec] [remaining 0min19sec] downloading band 11 ◑

DEBUG:urllib3.connectionpool:https://zipper.dataspace.copernicus.eu:443 "GET /odata/v1/Products(302382f3-fb33-4a3a-a5db-5836451b374e)/Nodes(S2B_MSIL2A_20230707T100559_N0509_R022_T32TPR_20230707T131302.SAFE)/Nodes(GRANULE)/Nodes(L2A_T32TPR_A033081_20230707T101428)/Nodes(IMG_DATA)/Nodes(R20m)/Nodes(T32TPR_20230707T100559_B12_20m.jp2)/$value HTTP/1.1" 200 33832340


 download products [ 98%] [elapsed 2min37sec] [remaining 0min03sec]:downloading band 12 ◕

ERROR[2023-12-08T19:51:06.917] multiprocess_manager.multi_download_file[2353] Error proc 0-['mystorage/downloaded_images/L2A_T32TPR_A033081_20230707T101428_2023-07-07/T32TPR_A033081_20230707T101428_B12.jp2']
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): identity.dataspace.copernicus.eu:443
DEBUG:urllib3.connectionpool:https://identity.dataspace.copernicus.eu:443 "DELETE /auth/realms/CDSE/account/sessions/048a4a12-1a2c-443a-b12a-5818625c2036 HTTP/1.1" 204 0
INFO[2023-12-08T19:51:07.013] download_products.download[857] end


 download products [100%] [elapsed 2min38sec]: ⬤['mystorage/downloaded_images/L2A_T32TPR_A033081_20230707T101428_2023-07-07']


## Data Preprocessing
To preprocess the images (converte from digital numbers to decimal values of reflectance) we use the function [**preprocess_products.preprocess**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.tools.preprocess_products.html#remotior_sensus.tools.preprocess_products.preprocess):
* `sensor` sets the sensor of input image, in this case `'Sentinel-2'`
* `input_path` accepts a directory containing the image bands, which in this case we get from `downloaded_image.extra['directory_paths'][0]`
* `output_path` sets output directory where converted bands are saved, in this case `output_path=working_directory + '/preprocessed'`
* `nodata_value` allows for masking image NoData by value.

Defining a list `preprocessed_image_directories` is convenient to retrieve the directories of preprocessed images that we are going to use later.
The process can take a few minutes.

In [10]:
preprocessed = rs.preprocess_products.preprocess(
    sensor="Sentinel-2",
    input_path=downloaded_image.extra["directory_paths"][0],
    output_path=working_directory + "/preprocessed",
    nodata_value=0,
)

INFO[2023-12-08T19:51:07.525] preprocess_products.perform_preprocess[137] start


 preprocess products [  0%]:starting ⚙

/opt/conda/envs/geo/lib/python3.11/site-packages/remotior_sensus/core/processor.py:536: RuntimeWarning: invalid value encountered in cast
  value_as_nodata = np.asarray(value_as_nodata).astype(
/opt/conda/envs/geo/lib/python3.11/site-packages/remotior_sensus/core/processor.py:536: RuntimeWarning: invalid value encountered in cast
  value_as_nodata = np.asarray(value_as_nodata).astype(


 preprocess products [ 89%] [elapsed 4min13sec] [remaining 0min31sec]:processing ⬤

INFO[2023-12-08T19:55:22.363] preprocess_products.perform_preprocess[628] end; preprocess products: ['mystorage/preprocessed/T32TPR_A033081_20230707T101428_B02.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B03.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B04.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B05.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B06.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B07.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B08.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B11.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B12.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B8A.tif']


 preprocess products [100%] [elapsed 4min14sec]: ⬤ining 0min31sec] processing ⚙

# 1 Create the Input Bandset



Now, we create a [**BandSet Catalog**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.core.bandset_catalog.html) which contains BandSets (note that an empty BandSet 1 will be automatically created).

In [11]:
catalog = rs.bandset_catalog()

INFO[2023-12-08T19:55:22.377] bandset_catalog.create[744] start
INFO[2023-12-08T19:55:22.380] bandset_catalog.create[751] end


Now we create a BandSet using `catalog.create_bandset`:
* `paths` defines the list of paths to the bands, in this case the directory of preprocessed files `[working_directory + '/preprocessed']` can be used to get all the bands inside this directory
* `bandset_number` explicitly sets the number of the BandSet as 1
* `wavelengths` sets a listo of values as the center wavelength for each band, in this case `['Sentinel-2']` automatically defines Sentinel-2 values for the identified bands in the BandSet

Optionally, we can set the `box_coordinate_list` argument as list of coordinates [left, top, right, bottom] to create a virtual subset and reduce computation time for a smaller area of interest.

In [12]:
catalog.create_bandset(
    paths=[working_directory + "/preprocessed"],
    bandset_number=1,
    wavelengths=["Sentinel-2"],
    box_coordinate_list=[629000, 5036000, 641000, 5028000],
)

INFO[2023-12-08T19:55:22.393] bandset_catalog.create[744] start
INFO[2023-12-08T19:55:22.983] bandset_catalog.create[948] end; file list: ['mystorage/preprocessed/T32TPR_A033081_20230707T101428_B02.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B03.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B04.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B05.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B06.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B07.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B08.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B11.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B12.tif', 'mystorage/preprocessed/T32TPR_A033081_20230707T101428_B8A.tif']


We get the first BandSet in the catalog with the function `catalog.get(1)`, then we can print the created BandSet with the BandSet function `print()`. This is the input that is going to be classified.

In [13]:
catalog.get(1).print()

name: T32TPR_A033081_20230707T101428_B 
date: NaT 
root directory: None 
crs: PROJCS["WGS 84 / UTM zone 32N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",9],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32632"]] 
box coordinate left │  629000 
box coordinate top │  5036000 
box coordinate right │  641000 
box coordinate bottom │  5028000 
┌─────────────┬─────────────┬───────────────────────────────────────────────────────────────┬────────────────┬────────────────────────────────────┬────────────┬─────────────────┬─────────────────┬

We can create a virtual raster of the BandSet which we can open in GIS software.

In [14]:
vrt_file = working_directory + "/virtual.vrt"
catalog.create_virtual_raster(bandset_number=1, output_path=vrt_file)

'mystorage/virtual.vrt'

We can display the virtual raster (using matplotlib).

In [ ]:
# @title Figure
from osgeo import gdal
import numpy
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

raster = gdal.Open(vrt_file)
blue_band = raster.GetRasterBand(1).ReadAsArray()
green_band = raster.GetRasterBand(2).ReadAsArray()
red_band = raster.GetRasterBand(3).ReadAsArray()
image = numpy.stack((red_band, green_band, blue_band), axis=-1)
image = numpy.clip((image / 10000) / 0.2, 0, 1)
plt.imshow(image)
plt.rcParams["figure.figsize"] = (15, 15)
plt.axis("off")
plt.show()
band = image = plt = None

# 2 Load the Training Input

In general, we would need to create a vector file containing polygons (ROIs) used to train the algorithm, where each polygon corresponds to a class (or macroclass).

In this tutorial, the following macroclasses are considered.


|   Macroclass name   |   Macroclass ID   |
|---------------------|-------------------|
|   Water             |   1               |
|   Built-up          |   2               |
|   Vegetation        |   3               |
|   Soil              |   4               |

For the creation of these polygons, one can use GIS software such as the [Semi-Automatic Classification Plugin](https://semiautomaticclassificationmanual.readthedocs.io) for QGIS.
For the purpose of this tutorial, we are going to download an already prepared geopackage of polygons.

> More information about **classes and macroclasses** is available [here](https://semiautomaticclassificationmanual.readthedocs.io/it/latest/remote_sensing.html#classes-and-macroclasses).


To open the training input, first we create a **Spectral Signatures Catalog** using [**SpectralSignaturesCatalog**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.core.spectral_signatures.html#remotior_sensus.core.spectral_signatures.SpectralSignaturesCatalog), which must be created specifically for a BandSet (and the characteristics thereof):


* `bandset` sets the reference BandSet



In [16]:
bandset_1 = catalog.get(1)

signature_catalog = rs.spectral_signatures_catalog(bandset=bandset_1)

In this case we are going to create the training polygons using the region growing at specific coordinates, but one can import a vector file ([read this](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.core.spectral_signatures.html#remotior_sensus.core.spectral_signatures.SpectralSignaturesCatalog.import_vector)).

In [17]:
region_path = rs.shared_tools.region_growing_polygon(
    coordinate_x=631837,
    coordinate_y=5034612,
    input_bands=1,
    band_number=1,
    max_width=100,
    max_spectral_distance=0.05,
    minimum_size=10,
    bandset_catalog=catalog,
)
signature_catalog.import_vector(
    file_path=region_path,
    macroclass_value=1,
    class_value=1,
    macroclass_name="water",
    class_name="water",
)
region_path = rs.shared_tools.region_growing_polygon(
    coordinate_x=631859,
    coordinate_y=5032672,
    input_bands=1,
    band_number=1,
    max_width=100,
    max_spectral_distance=0.05,
    minimum_size=10,
    bandset_catalog=catalog,
)
signature_catalog.import_vector(
    file_path=region_path,
    macroclass_value=2,
    class_value=2,
    macroclass_name="built-up",
    class_name="built-up",
)
region_path = rs.shared_tools.region_growing_polygon(
    coordinate_x=630552,
    coordinate_y=5029558,
    input_bands=1,
    band_number=1,
    max_width=100,
    max_spectral_distance=0.05,
    minimum_size=10,
    bandset_catalog=catalog,
)
signature_catalog.import_vector(
    file_path=region_path,
    macroclass_value=3,
    class_value=3,
    macroclass_name="vegetation",
    class_name="vegetation",
)
region_path = rs.shared_tools.region_growing_polygon(
    coordinate_x=631226,
    coordinate_y=5028546,
    input_bands=1,
    band_number=1,
    max_width=100,
    max_spectral_distance=0.05,
    minimum_size=10,
    bandset_catalog=catalog,
)
signature_catalog.import_vector(
    file_path=region_path,
    macroclass_value=4,
    class_value=4,
    macroclass_name="soil",
    class_name="soil",
)

 remotior_sensus [  0%]:starting ⚙

Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195529805003_8382 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.


 remotior_sensus [ 57%]:calculate signature ⚙

Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195543305856_548 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.


 remotior_sensus [ 57%] calculate signature ⚙

Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195554715835_3024 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.


 remotior_sensus [ 57%]:calculate signature ⚙

Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195607767695_3922 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.


 remotior_sensus [ 57%]:calculate signature ⚙

# 3 Perform Random Forest Classification

We are going to train the **Random Forest** classifier and perform the classification.

To classify the image we are going to use the tool [**band_classification**](https://remotior-sensus.readthedocs.io/en/latest/remotior_sensus.tools.band_classification.html#remotior_sensus.tools.band_classification.band_classification), with the following parameters:
* `input_bands` is a parameter that accepts a list of input raster paths, or a BandSet number, or a previously defined BandSet
* `bandset_catalog` the catalog containing the BandSet (in case `input_bands` is a BandSet number)
* `spectral_signatures` the SpectralSignaturesCatalog containing the training input
* `macroclass` if True, the training in evaluated on the macroclass ID field; if False, the training is evaluated on the class ID field
* `output_path` sets the output path of the classification raster
* `save_classifier` is an optional parameter, if True saves the classifier to a file to be loaded (without training) for a new classification
* `classification_confidence` is an optional parameter, if True, writes a classification confidence raster as additional output, useful to evaluate the confidence of the classifier at pixel level (from 0=low to 1=high confidence)
* `algorithm_name` is the name of the classification algorithms, which for Random Forest is `'rf'`, other options are:
  * Minimum Distance = 'md'
  * Maximum Likelihood = 'ml'
  * Spectral Angle Mapping = 'sam'
  * Random Forest OneVsRest = 'rf_ovr'
  * Support Vector Machine = 'svm'
  * Multi Layer Perceptron = 'mlp'
  * Multi Layer Perceptron using PyTorch framework = 'pytorch_mlp'

    classification_confidence=True
Available parameters for **Random Forest** are:
* `rf_max_features` parameter that sets the number of features considered in node splitting; if None all features are considered in node splitting, available options are ‘sqrt’ as square root of all the features, an integer number, or a float number for a fraction of all the features.
* `rf_number_trees` parameter that sets the number of trees in the forest
* `rf_min_samples_split` parameter that sets the minimum number of samples required to split an internal node

For the purpose of this tutorial we set `rf_number_trees=50` to reduce computation time, but generally it should be at least `rf_number_trees=200` to get good results.

In [18]:
classification = rs.band_classification(
    input_bands=1,
    bandset_catalog=catalog,
    spectral_signatures=signature_catalog,
    macroclass=True,
    output_path=working_directory + "/classification_rf.tif",
    classification_confidence=True,
    algorithm_name="rf",
    rf_number_trees=50,
    rf_min_samples_split=2,
)

INFO[2023-12-08T19:56:16.222] band_classification.band_classification[1255] start


 band classification [  1%] [elapsed 0min00sec] [remaining 0min00sec]:starting the classifier ⚙

Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not nor

 band classification [  1%] [elapsed 0min06sec] [remaining 0min00sec] get band arrays ○ier ⚙

Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type POLYGON is inserted into layer t20231208_195524861828_8682 of geometry type MULTIPOLYGON, which is not nor

...

 band classification [ 20%] [elapsed 0min51sec] [remaining 1min47sec]:fitting ⬤

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    2.1s


 band classification [ 81%] [elapsed 0min57sec] [remaining 0min10sec]:writing raster ⚙

INFO[2023-12-08T19:57:15.364] band_classification.band_classification[1358] end; prediction: mystorage/classification_rf.tif


 band classification [ 85%] [elapsed 0min59sec] [remaining 0min10sec]:writing raster ⚙

We can display the resulting classification raster.

In [ ]:
# @title Figure
from osgeo import gdal
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

image = gdal.Open(classification.path)
band = image.GetRasterBand(1).ReadAsArray()
color_map = mcolors.ListedColormap(["blue", "red", "green", "yellow"])
plt.imshow(band, cmap=color_map, vmin=1, vmax=4)
plt.rcParams["figure.figsize"] = (15, 15)
plt.axis("off")
legend_handles = [
    mpatches.Patch(color=c, label=str(val))
    for val, c in zip(
        ["water", "built-up", "vegetation", "soil"], ["blue", "red", "green", "yellow"]
    )
]
plt.legend(
    handles=legend_handles, title="Classes", bbox_to_anchor=(0, 1), loc="upper left"
)
plt.show()
band = image = plt = None

We can also display the **confidence raster** of the classification, which is a raster having values ranging from 0 (low confidence) to 1 (high confidence).

We can use this confidence raster to assess the quality of the classification and collect additional training areas to improve the results.

In [ ]:
# @title Figure
from osgeo import gdal
import numpy
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

image = gdal.Open(classification.extra["algorithm_raster"])
band = image.GetRasterBand(1).ReadAsArray()
plt.imshow(band, cmap="viridis", vmin=0, vmax=1)
plt.rcParams["figure.figsize"] = (15, 15)
plt.axis("off")

color_values = numpy.linspace(0, 1, 5)
color_map = plt.cm.viridis
legend_handles = []
for v in color_values:
    legend_handles.append(mpatches.Patch(color=color_map(v), label=str(v)))
plt.legend(
    handles=legend_handles, title="Confidence", bbox_to_anchor=(0, 1), loc="upper left"
)

plt.show()
band = image = plt = None

# Close the Session
The session should be closed at the end of all the processes to **remove the temporary files and stop subprocesses**.

In [21]:
rs.close()